# 📊 UMKM Analytics - Data Pipeline
**100% FREE - Menggunakan Google Colab + BigQuery**

Notebook ini menggantikan Cloud Functions yang memerlukan billing.

## Fitur:
- Upload data lokal ke BigQuery
- ETL Pipeline
- Generate Daily Summary
- Sentiment Analysis dari Tokopedia Reviews

## 1️⃣ Setup & Authentication

In [ ]:
# Install dependencies
!pip install -q google-cloud-bigquery pandas pyarrow db-dtypes

In [ ]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

print('✅ Authenticated successfully!')

In [ ]:
# Configuration
PROJECT_ID = 'ipsd-483408'  # Ganti dengan Project ID Anda
DATASET_ID = 'umkm_analytics'
LOCATION = 'asia-southeast2'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

print(f'✅ Connected to BigQuery: {PROJECT_ID}')

## 2️⃣ Upload Sample Data ke BigQuery

In [ ]:
import pandas as pd

# Upload file dari local atau Google Drive
from google.colab import files

print('📁 Upload file transactions.csv')
uploaded = files.upload()

In [ ]:
# Load dan preview data
import io

# Ambil nama file yang diupload
filename = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[filename]))

print(f'📊 Total Records: {len(df)}')
print(f'📋 Columns: {df.columns.tolist()}')
df.head()

In [ ]:
# Upload ke BigQuery
table_id = f'{PROJECT_ID}.{DATASET_ID}.raw_sales'

# Configure job
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Replace existing
    autodetect=True,
)

# Upload
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()  # Wait for completion

table = client.get_table(table_id)
print(f'✅ Uploaded {table.num_rows} rows to {table_id}')

## 3️⃣ Upload Tokopedia Reviews

In [ ]:
print('📁 Upload file tokopedia_product_reviews_2025.csv')
uploaded_reviews = files.upload()

In [ ]:
# Load reviews
review_file = list(uploaded_reviews.keys())[0]
df_reviews = pd.read_csv(io.BytesIO(uploaded_reviews[review_file]))

print(f'📊 Total Reviews: {len(df_reviews)}')
print(f'📋 Columns: {df_reviews.columns.tolist()}')

# Sentiment distribution
print('\n🎭 Sentiment Distribution:')
print(df_reviews['sentiment_label'].value_counts())

In [ ]:
# Upload reviews ke BigQuery
reviews_table_id = f'{PROJECT_ID}.{DATASET_ID}.tokopedia_reviews'

job = client.load_table_from_dataframe(df_reviews, reviews_table_id, job_config=job_config)
job.result()

table = client.get_table(reviews_table_id)
print(f'✅ Uploaded {table.num_rows} reviews to {reviews_table_id}')

## 4️⃣ Generate Daily Summary (ETL)

In [ ]:
# ETL Query: Generate Daily Summary
etl_query = f'''
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.daily_summary` AS
SELECT 
    sale_date as summary_date,
    COUNT(transaction_id) as total_transactions,
    SUM(total_amount) as total_revenue,
    SUM(quantity) as total_quantity,
    ROUND(AVG(total_amount), 0) as avg_order_value,
    ARRAY_AGG(category ORDER BY total_amount DESC LIMIT 1)[OFFSET(0)] as top_category,
    ARRAY_AGG(product_name ORDER BY total_amount DESC LIMIT 1)[OFFSET(0)] as top_product,
    COUNT(DISTINCT seller_name) as unique_sellers,
    CURRENT_TIMESTAMP() as created_at
FROM `{PROJECT_ID}.{DATASET_ID}.raw_sales`
GROUP BY sale_date
ORDER BY sale_date DESC
'''

job = client.query(etl_query)
job.result()

print('✅ Daily summary generated!')

In [ ]:
# Preview daily summary
query = f'''
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.daily_summary`
ORDER BY summary_date DESC
LIMIT 10
'''

df_summary = client.query(query).to_dataframe()
df_summary

## 5️⃣ Analisis Data

In [ ]:
# Top Categories
query = f'''
SELECT 
    category,
    COUNT(*) as transactions,
    SUM(total_amount) as revenue
FROM `{PROJECT_ID}.{DATASET_ID}.raw_sales`
GROUP BY category
ORDER BY revenue DESC
'''

df_categories = client.query(query).to_dataframe()

# Visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(df_categories['category'], df_categories['revenue']/1000000)
plt.xlabel('Revenue (Juta Rupiah)')
plt.title('Revenue per Kategori UMKM')
plt.tight_layout()
plt.show()

In [ ]:
# Sentiment Analysis dari Tokopedia
query = f'''
SELECT 
    sentiment_label,
    COUNT(*) as count,
    ROUND(AVG(rating), 2) as avg_rating
FROM `{PROJECT_ID}.{DATASET_ID}.tokopedia_reviews`
GROUP BY sentiment_label
'''

df_sentiment = client.query(query).to_dataframe()

# Pie chart
plt.figure(figsize=(8, 8))
colors = ['#4CAF50', '#FFC107', '#F44336']
plt.pie(df_sentiment['count'], labels=df_sentiment['sentiment_label'], 
        autopct='%1.1f%%', colors=colors, startangle=90)
plt.title('Sentiment Distribution - Tokopedia Reviews')
plt.show()

## ✅ Selesai!

Data sudah tersimpan di BigQuery dan siap untuk:
- Dibuat dashboard di Looker Studio
- Dianalisis lebih lanjut
- Di-export ke format lain